In [8]:
import os
import json
import numpy as np
import pandas as pd
import requests
import datetime

from dotenv import load_dotenv
from google.cloud import storage

In [9]:
load_dotenv()
api_key = os.getenv("CBIBS_API_KEY")
if not api_key:
    raise ValueError("No API key found. Please check your .env file.")

The CBIBS API takes a handful of parameters:
- `key` (required): Your API key
- `var`: Variable required, can be `all`.
- `sd`: Start date, ISO8601 format (e.g. `2024-07-05T00:00:00Z`)
- `ed`: End date, ISO8601 format (e.g. `2024-07-06T01:00:00Z`)

An example query would look like this:
```
https://mw.buoybay.noaa.gov/api/v1/json/query/AN?key=<<api_key>>&sd=2020-04-01T10:00:00z&ed=2020-04-01T20:00:00z&var=sea_water_temperature
```

I'm most interested in Potomac, the `PL` station.

In [21]:
def concat_url(key, station, sd, var='all', ed=f'{datetime.datetime.now().isoformat()}Z'):
    url = f"https://mw.buoybay.noaa.gov/api/v1/json/query/{station}?key={key}&sd={sd}&var={var}"
    print(url)
    return

concat_url(api_key, 'PL', sd='2024-07-01T00:00:00Z')

https://mw.buoybay.noaa.gov/api/v1/json/query/PL?key=f159959c117f473477edbdf3245cc2a4831ac61f&sd=2024-07-01T00:00:00Z&ed=2024-07-06T14:28:36.197918Z&var=all


In [6]:
def query_cbibs(key, station, var='all', sd, ed=None):
    api_url = f"https://mw.buoybay.noaa.gov/api/v1/json/query/{station}?key={key}&sd={sd}&ed={ed}&var={var}"
    print(api_url)
    response = requests.get(api_url)
    data = response.json()
    df = pd.DataFrame(data)
    return df

f159959c117f473477edbdf3245cc2a4831ac61f
